In [1]:
from pulp import *
import pandas as pd

### Make DataFrames from excel files
index_col = 0, means the first column of the dataframe is used as row lable
squeeze = 'TRUE', if parsed data has only one column then return it a series

In [6]:
AC=pd.read_excel('LAGD_wlp.xlsx', sheet_name = 'assign_cost', index_col = 0)
FC=pd.read_excel('LAGD_wlp.xlsx', sheet_name = 'f_cost', index_col = 0, squeeze = 'TRUE')

C:\Users\ehdwn\AppData\Local\Temp\ipykernel_12068\3843704639.py:2: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  FC=pd.read_excel('LAGD_wlp.xlsx', sheet_name = 'f_cost', index_col = 0, squeeze = 'TRUE')


In [7]:
AC

,0,1,2,3,4,5,6
A,1,2,10,9,6,7,3
B,2,9,0,7,3,6,10
C,7,6,1,5,3,10,5
D,6,5,10,2,6,3,6
E,6,4,6,3,7,2,6


In [8]:
FC

locations
A    5
B    7
C    5
D    6
E    5
Name: fi, dtype: int64

### Convert the DataFrame to dictionary
orient="index", convert like {index -> {column -> value}}, for example 'ac' below

In [9]:
ac=AC.to_dict(orient="index")
fc=FC.to_dict()

In [10]:
ac

{'A': {0: 1, 1: 2, 2: 10, 3: 9, 4: 6, 5: 7, 6: 3},
 'B': {0: 2, 1: 9, 2: 0, 3: 7, 4: 3, 5: 6, 6: 10},
 'C': {0: 7, 1: 6, 2: 1, 3: 5, 4: 3, 5: 10, 6: 5},
 'D': {0: 6, 1: 5, 2: 10, 3: 2, 4: 6, 5: 3, 6: 6},
 'E': {0: 6, 1: 4, 2: 6, 3: 3, 4: 7, 5: 2, 6: 6}}

In [11]:
fc

{'A': 5, 'B': 7, 'C': 5, 'D': 6, 'E': 5}

### Convert the index or column from the DataFrame to list

In [17]:
I=AC.index.tolist()
J=AC.columns.tolist()

In [18]:
I

['A', 'B', 'C', 'D', 'E']

In [19]:
J

[0, 1, 2, 3, 4, 5, 6]

In [21]:
model = pulp.LpProblem("Plant_Location_Problem", LpMinimize)

In [22]:
x = LpVariable.dicts('x', (I,J), 0, 1, LpBinary)
y = LpVariable.dicts('y', (I), 0, 1, LpBinary)

In [23]:
x

{'A': {0: x_A_0, 1: x_A_1, 2: x_A_2, 3: x_A_3, 4: x_A_4, 5: x_A_5, 6: x_A_6},
 'B': {0: x_B_0, 1: x_B_1, 2: x_B_2, 3: x_B_3, 4: x_B_4, 5: x_B_5, 6: x_B_6},
 'C': {0: x_C_0, 1: x_C_1, 2: x_C_2, 3: x_C_3, 4: x_C_4, 5: x_C_5, 6: x_C_6},
 'D': {0: x_D_0, 1: x_D_1, 2: x_D_2, 3: x_D_3, 4: x_D_4, 5: x_D_5, 6: x_D_6},
 'E': {0: x_E_0, 1: x_E_1, 2: x_E_2, 3: x_E_3, 4: x_E_4, 5: x_E_5, 6: x_E_6}}

In [24]:
y

{'A': y_A, 'B': y_B, 'C': y_C, 'D': y_D, 'E': y_E}

In [25]:
model += lpSum (fc[i] *y[i] for i in I)+  pulp.lpSum(ac [i][j] * x[i][j] for i in I for j in J )," objective function "

In [26]:
for j in J :
    model += lpSum(x[i][j] for i in I ) == 1,f"satisfy demand {j}"
    
for j in J:
    for i in I:
        model += x[i][j] <= y[i] ,f"serve from existing facilities {i}-{j}"

In [27]:
model.writeLP("Plant_Location_Problem")

[x_A_0,
 x_A_1,
 x_A_2,
 x_A_3,
 x_A_4,
 x_A_5,
 x_A_6,
 x_B_0,
 x_B_1,
 x_B_2,
 x_B_3,
 x_B_4,
 x_B_5,
 x_B_6,
 x_C_0,
 x_C_1,
 x_C_2,
 x_C_3,
 x_C_4,
 x_C_5,
 x_C_6,
 x_D_0,
 x_D_1,
 x_D_2,
 x_D_3,
 x_D_4,
 x_D_5,
 x_D_6,
 x_E_0,
 x_E_1,
 x_E_2,
 x_E_3,
 x_E_4,
 x_E_5,
 x_E_6,
 y_A,
 y_B,
 y_C,
 y_D,
 y_E]

In [28]:
model.solve()

1

In [30]:
for  var in  model.variables():
    if var.varValue >0:
        print(var.name ,"=", var.varValue)

x_A_0 = 1.0
x_A_1 = 1.0
x_A_6 = 1.0
x_C_2 = 1.0
x_C_4 = 1.0
x_E_3 = 1.0
x_E_5 = 1.0
y_A = 1.0
y_C = 1.0
y_E = 1.0
